In [102]:
# imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import joblib

plt.style.use("dark_background")
pd.pandas.set_option("display.max_columns", None)

### Data import

In [103]:
# load the data set
data = pd.read_csv("train.csv")

print(data.shape)

data.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Separate in train and test set

In [104]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['Id', 'SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.1,
    random_state=87
)

print(X_train.shape, X_test.shape)

(1314, 79) (146, 79)


## Feature Engineering

### Targets

In [105]:
# transform targets to log scake
y_train, y_test = np.log(y_train), np.log(y_test)

### Missing values

In [106]:
# Categorical variables
cat_vars = [
    var for var in data.columns if data[var].dtype == 'object'
]
cat_vars += ['MSSubClass']

X_train[cat_vars] = X_train[cat_vars].astype('object')
X_test[cat_vars] = X_test[cat_vars].astype('object')

# categories with missing data
cat_vars_with_nan = [
    var for var in cat_vars
    if X_train[var].isna().sum() > 0
]


# Split in >10% missing and <10% missing
with_string_missing = [
    var for var in cat_vars_with_nan
    if X_train[var].isna().mean() > 0.1
]

with_frequent_category = [
    var for var in cat_vars_with_nan
    if X_train[var].isna().mean() < 0.1
]

# replace missing values with missing string
X_train[with_string_missing] = (
    X_train[with_string_missing]
    .fillna('Missing')
)
X_test[with_string_missing] = (
    X_test[with_string_missing]
    .fillna('Missing')
)

# replace missing values with most frequent value
for var in with_frequent_category:
    mode = X_train[var].mode()[0]
    
    print(var, mode)
    
    X_train[var] = X_train[var].fillna(mode)
    X_test[var] = X_test[var].fillna(mode)

BsmtQual TA
BsmtCond TA
BsmtExposure No
BsmtFinType1 Unf
BsmtFinType2 Unf
GarageType Attchd
GarageFinish Unf
GarageQual TA
GarageCond TA


In [107]:
# numerical variables
num_vars = [
    var for var in X_train.columns
    if var not in cat_vars
    and var != 'SalePrice'
]

print(len(num_vars))

# numerical variables with Nan
vars_with_nan = [
    var for var in num_vars
    if X_train[var].isna().sum() > 0
]

print(100 * X_train[vars_with_nan].isna().mean())

# add binary missing indicator
# impute missing value with the mean
for var in vars_with_nan:
    mean_val = X_train[var].mean()
    
    print(var, mean_val)
    
    # binary indicator col
    X_train[var + '_na'] = np.where(
        X_train[var].isna(), 1, 0
    )
    X_test[var + '_na'] = np.where(
        X_test[var].isna(), 1, 0
    )
    
    # replace with mean value
    X_train[var] = X_train[var].fillna(mean_val)    
    X_test[var] = X_test[var].fillna(mean_val)

# check for NaNs
X_train[vars_with_nan].isna().sum()

35
LotFrontage    17.351598
MasVnrArea      0.532725
GarageYrBlt     5.859970
dtype: float64
LotFrontage 70.2219152854512
MasVnrArea 101.45600612088752
GarageYrBlt 1978.3371059013743


LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

### Temporal feature engineering

In [108]:
# temporal variable feature engineering
def elapsed_years(df, var):
    df[var] = df['YrSold'] - df[var]
    return df

# apply elapsed years
for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    X_train = elapsed_years(X_train, var)
    X_test = elapsed_years(X_test, var)
    
# drop original year column
X_train = X_train.drop(['YrSold'], axis=1)
X_test = X_test.drop(['YrSold'], axis=1)

### Numerical feature engineering

In [109]:
# logaritmic feature engineering
for var in ['LotFrontage', '1stFlrSF', 'GrLivArea']:
    X_train[var] = np.log(X_train[var])
    X_test[var] = np.log(X_test[var])

# Yeo-Johnson transformation
X_train['LotArea'], param = stats.yeojohnson(X_train['LotArea'])
X_test['LotArea'] = stats.yeojohnson(X_test['LotArea'], lmbda=param)

print(param)

# binarize skewed variables
skewed = ['BsmtFinSF2', 'LowQualFinSF', 'EnclosedPorch',
          '3SsnPorch', 'ScreenPorch', 'MiscVal']

for var in skewed:
    X_train[var] = np.where(X_train[var] == 0, 0, 1)
    X_test[var] = np.where(X_test[var] == 0, 0, 1)

-0.0007736202777277588


### Categorical feature engineering

In [110]:
# Map the quality categories to ordinal values
qual_mappings = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'Missing': 0, 'NA': 0}

qual_vars = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
             'HeatingQC', 'KitchenQual', 'FireplaceQu',
             'GarageQual', 'GarageCond',
            ]

for var in qual_vars:
    X_train[var] = X_train[var].map(qual_mappings)
    X_test[var] = X_test[var].map(qual_mappings)
    
# Map the expose categories to ordinal values
exposure_mappings = {'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4, 'Missing': 0, 'NA': 0}

var = 'BsmtExposure'

X_train[var] = X_train[var].map(exposure_mappings)
X_test[var] = X_test[var].map(exposure_mappings)

# Map the finish categories to ordinal values
finish_mappings = {'Missing': 0, 'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}

finish_vars = ['BsmtFinType1', 'BsmtFinType2']

for var in finish_vars:
    X_train[var] = X_train[var].map(finish_mappings)
    X_test[var] = X_test[var].map(finish_mappings)
    
# Map the garage categories to ordinal values
garage_mappings = {'Missing': 0, 'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}

var = 'GarageFinish'

X_train[var] = X_train[var].map(garage_mappings) 
X_test[var] = X_test[var].map(garage_mappings) 

# Map the fence categories to ordinal values
fence_mappings = {'Missing': 0, 'NA': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}

var = 'Fence'

X_train[var] = X_train[var].map(fence_mappings)
X_test[var] = X_test[var].map(fence_mappings)

### Remove rare labels

In [111]:
# combine quality variables
qual_vars += finish_vars + ['BsmtExposure',
                            'GarageFinish',
                            'Fence']

# capture rare variables
cat_others = [
    var for var in cat_vars if var not in qual_vars
]

print(len(cat_others))

30


In [112]:
# find labels that are more frequent than a given percentage
def find_frequent_labels(df, var, rare_perc):
    df = df.copy()
    
    tmp = df.groupby(var)[var].count() / len(df)
    
    return tmp[tmp > rare_perc].index

# find frequent categories
for var in cat_others:
    frequent_ls = find_frequent_labels(X_train, var, 0.01)
    
    print(var, frequent_ls, sep='\n')
    
    # label rare categories with Rare
    X_train[var] = np.where(X_train[var].isin(
        frequent_ls), X_train[var], 'Rare')
    X_test[var] = np.where(X_test[var].isin(
        frequent_ls), X_test[var], 'Rare')

MSZoning
Index(['FV', 'RL', 'RM'], dtype='object', name='MSZoning')
Street
Index(['Pave'], dtype='object', name='Street')
Alley
Index(['Grvl', 'Missing', 'Pave'], dtype='object', name='Alley')
LotShape
Index(['IR1', 'IR2', 'Reg'], dtype='object', name='LotShape')
LandContour
Index(['Bnk', 'HLS', 'Low', 'Lvl'], dtype='object', name='LandContour')
Utilities
Index(['AllPub'], dtype='object', name='Utilities')
LotConfig
Index(['Corner', 'CulDSac', 'FR2', 'Inside'], dtype='object', name='LotConfig')
LandSlope
Index(['Gtl', 'Mod'], dtype='object', name='LandSlope')
Neighborhood
Index(['Blmngtn', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor',
       'Edwards', 'Gilbert', 'IDOTRR', 'Mitchel', 'NAmes', 'NWAmes', 'NoRidge',
       'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst',
       'StoneBr', 'Timber'],
      dtype='object', name='Neighborhood')
Condition1
Index(['Artery', 'Feedr', 'Norm', 'PosN', 'RRAn'], dtype='object', name='Condition1')
Condition2
Index(['Norm'], dty

In [113]:
# map the categorical values to ordinal values
def replace_cats(train, test, y_train, var, target):
    tmp = pd.concat([X_train, y_train], axis=1)
    
    sorted_labels = (tmp
                     .groupby([var])[target]
                     .mean()
                     .sort_values()
                     .index
    )
    
    ordinal_labels = {k: i for i, k in enumerate(sorted_labels, 0)}
    
    print(var, ordinal_labels, sep='\n')
    
    train[var] = train[var].map(ordinal_labels)
    test[var] = test[var].map(ordinal_labels)

for var in cat_others:
    replace_cats(X_train, X_test, y_train, var, 'SalePrice')

MSZoning
{'Rare': 0, 'RM': 1, 'RL': 2, 'FV': 3}
Street
{'Rare': 0, 'Pave': 1}
Alley
{'Grvl': 0, 'Pave': 1, 'Missing': 2}
LotShape
{'Reg': 0, 'IR1': 1, 'Rare': 2, 'IR2': 3}
LandContour
{'Bnk': 0, 'Lvl': 1, 'Low': 2, 'HLS': 3}
Utilities
{'Rare': 0, 'AllPub': 1}
LotConfig
{'FR2': 0, 'Inside': 1, 'Corner': 2, 'Rare': 3, 'CulDSac': 4}
LandSlope
{'Gtl': 0, 'Mod': 1, 'Rare': 2}
Neighborhood
{'IDOTRR': 0, 'BrDale': 1, 'BrkSide': 2, 'OldTown': 3, 'Edwards': 4, 'Sawyer': 5, 'SWISU': 6, 'NAmes': 7, 'Rare': 8, 'Mitchel': 9, 'SawyerW': 10, 'NWAmes': 11, 'Gilbert': 12, 'CollgCr': 13, 'Blmngtn': 14, 'Crawfor': 15, 'ClearCr': 16, 'Somerst': 17, 'Timber': 18, 'StoneBr': 19, 'NridgHt': 20, 'NoRidge': 21}
Condition1
{'Artery': 0, 'Feedr': 1, 'Norm': 2, 'RRAn': 3, 'Rare': 4, 'PosN': 5}
Condition2
{'Rare': 0, 'Norm': 1}
BldgType
{'2fmCon': 0, 'Duplex': 1, 'Twnhs': 2, '1Fam': 3, 'TwnhsE': 4}
HouseStyle
{'SFoyer': 0, '1.5Fin': 1, 'Rare': 2, '1Story': 3, 'SLvl': 4, '2Story': 5}
RoofStyle
{'Gable': 0, 'Rare': 

### Feature scaling

In [114]:
# crate scaler
scaler = MinMaxScaler()

# fit the scaler
scaler.fit(X_train)

# apply the scaler and transform the variables
X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns
)
X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns
)

X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
0,0.333333,0.666667,0.490408,0.372646,1.0,1.0,0.000000,0.333333,1.0,0.25,0.0,0.333333,0.0,1.0,0.75,0.2,0.444444,0.500,0.485294,0.950820,0.0,0.0,0.2,0.4,0.333333,0.000,0.333333,0.50,0.50,0.333333,0.666667,0.000000,0.4,0.065202,0.6,1.0,0.153682,0.125696,1.0,0.5,1.0,1.000000,0.402688,0.197579,0.0,0.500838,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.333333,0.333333,1.0,0.333333,0.8,0.75,0.616822,0.0,0.25,0.169252,0.5,0.5,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.272727,0.666667,0.75,0.0,0.0,0.0
1,0.000000,0.666667,0.328426,0.276953,1.0,1.0,0.333333,0.000000,1.0,0.25,0.0,0.095238,0.4,1.0,0.75,0.6,0.333333,0.750,0.610294,0.803279,0.0,0.0,0.3,0.2,0.333333,0.000,0.333333,0.50,1.00,0.666667,0.666667,0.000000,0.0,0.000000,0.0,0.0,0.188356,0.072013,1.0,0.5,1.0,0.666667,0.361859,0.000000,0.0,0.338257,0.000000,0.0,0.333333,0.0,0.250,0.333333,0.000000,0.166667,1.0,0.000000,0.0,0.75,0.258532,0.0,0.00,0.000000,0.5,0.5,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.636364,0.666667,0.75,0.0,0.0,1.0
2,0.583333,0.666667,0.388581,0.317031,1.0,1.0,0.000000,1.000000,1.0,0.25,0.5,0.714286,0.4,1.0,0.75,1.0,0.666667,0.625,0.566176,0.147541,0.0,0.0,0.5,0.4,0.333333,0.000,0.333333,0.25,0.25,0.666667,0.666667,0.000000,0.0,0.000000,0.0,0.0,0.332192,0.127005,1.0,0.5,1.0,1.000000,0.353938,0.315254,0.0,0.531832,0.000000,0.0,0.333333,0.5,0.375,0.333333,0.333333,0.333333,1.0,0.333333,0.8,0.75,0.719626,0.5,0.25,0.190409,0.5,0.5,0.5,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.75,1.0,0.0,0.090909,0.666667,0.75,0.0,0.0,0.0
3,0.000000,0.666667,0.388581,0.348053,1.0,1.0,0.000000,0.000000,1.0,0.25,0.0,0.285714,0.4,1.0,0.75,0.6,0.111111,0.500,0.654412,0.983607,0.0,0.0,0.3,0.2,0.333333,0.000,0.333333,0.50,0.50,0.333333,0.333333,0.000000,0.4,0.051382,0.0,0.0,0.000000,0.047463,1.0,0.5,0.0,0.333333,0.102585,0.000000,0.0,0.095894,0.000000,0.0,0.333333,0.0,0.125,0.333333,0.000000,0.083333,1.0,0.000000,0.0,0.25,0.738318,0.0,0.25,0.173484,0.5,0.5,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.000000,0.666667,0.75,0.0,0.0,0.0
4,1.000000,0.666667,0.476078,0.374734,1.0,1.0,0.000000,0.333333,1.0,0.25,0.0,0.952381,0.4,1.0,0.75,1.0,0.777778,0.500,0.022059,0.049180,0.0,0.0,0.9,0.9,0.666667,0.215,0.666667,0.50,1.00,1.000000,0.666667,0.666667,0.0,0.000000,0.0,0.0,0.489298,0.187070,1.0,1.0,1.0,1.000000,0.465575,0.644068,0.0,0.708224,0.000000,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,1.0,0.333333,0.8,1.00,0.028037,0.5,0.75,0.600846,0.5,0.5,1.0,0.224037,0.300797,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.818182,0.666667,0.75,0.0,0.0,0.0


### Store transformed data sets and the scaler

In [115]:
X_train.to_csv('xtrain.csv', index=False)
X_test.to_csv('xtest.csv', index=False)

y_train.to_csv('ytrain.csv', index=False)
y_test.to_csv('ytest.csv', index=False)

joblib.dump(scaler, 'minmax_scaler.joblib')

['minmax_scaler.joblib']